# NIHDAM model 09/04/2018

In [1]:
from training_tools import *
from custom_hdam import CustomHDAMVAE, pad_batch
from custom_cnn_model import CustomCNN
import pickle

In [2]:
import gensim
nm=gensim.models.KeyedVectors.load_word2vec_format('./data/network_undirected.embd')
D=pickle.load(open('./data/docids_linkids_map_clean.pkl'))

In [3]:
HDAMAP=pickle.load(open('/mnt/sshd/vvkulkarni/docid_token_dict_for_hdam.pkl'))

In [4]:
def get_network_features(docid):
    if docid in D:
        linkids=D[docid]
        linkids_str=[unicode(l) for l in linkids] #80 is twitter
        if len(linkids_str):
            return nm[linkids_str].mean(axis=0)
        else:
            return np.zeros(shape=128)
    else:
        return np.zeros(128)

def get_network_features_batch(batch):
    docids=batch.docid.data.cpu().numpy()
    nfeatures=np.array([get_network_features(d) for d in docids])
    if use_cuda:
        return Variable(torch.from_numpy(nfeatures)).float().cuda()
    else:
        return Variable(torch.from_numpy(nfeatures)).float()

In [5]:
def get_hdam_features(docid):
    if docid in HDAMAP:
        return HDAMAP[docid]
    else:
        return [list(s) for s in np.zeros(shape=(10,10)).astype(int)]
    
def get_hdam_features_batch(batch):
    docids=batch.docid.data.cpu().numpy()
    nfeatures=pad_batch([get_hdam_features(d) for d in docids])
    if use_cuda:
        return nfeatures.cuda()
    else:
        return nfeatures

In [6]:
def get_examples(b):
    fnn_view = get_network_features_batch(b)
    hdam_view=get_hdam_features_batch(b)
    cnn_view=b.example.transpose(0,1)
    return (fnn_view,hdam_view, cnn_view)

def get_targets(b):
    return b.target

In [7]:
if use_cuda:
    feature_tensor_type=torch.cuda.LongTensor
    target_tensor_type=torch.cuda.LongTensor
else:
    feature_tensor_type=torch.LongTensor
    target_tensor_type=torch.LongTensor
INDEX=torchtext.data.Field(sequential=False, use_vocab=False)
TEXT=torchtext.data.ReversibleField(tensor_type=feature_tensor_type, fix_length=40)
TARGET=torchtext.data.Field(preprocessing=int, sequential=False, use_vocab=False, tensor_type=target_tensor_type)
SOURCE=torchtext.data.Field(preprocessing=int, sequential=False, use_vocab=False, tensor_type=target_tensor_type)
DOCID=torchtext.data.Field(preprocessing=int, sequential=False, use_vocab=False, tensor_type=target_tensor_type)

In [8]:
train_iterator, val_iterator, test_iterator=load_dataset_fake('./data/title_selected_sources_ids_with_targets_train.input', './data/title_selected_sources_ids_with_targets_val.input','./data/title_selected_sources_ids_with_targets_test_final.input', INDEX, TEXT, TARGET, SOURCE, DOCID, min_freq=1, batch_size=64)

In [9]:
from gensim.models.keyedvectors import KeyedVectors 
word_vectors = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True) 

In [10]:
wv_matrix = []                                                                                                                                                           
for i in range(len(TEXT.vocab)):                                                                                                                                      
    word = TEXT.vocab.itos[i]                                                                                                                                        
    if word in word_vectors.vocab:                                                                                                                                       
        wv_matrix.append(word_vectors.word_vec(word))                                                                                                                    
    else:  
        wv_matrix.append(np.random.uniform(-0.01, 0.01, 300).astype("float32"))                                                                                          

# one for UNK and one for zero padding                                                                                                                                   
wv_matrix.append(np.random.uniform(-0.01, 0.01, 300).astype("float32"))                                                                                                  
wv_matrix.append(np.zeros(300).astype("float32"))                                                                                                                        
wv_matrix = np.array(wv_matrix)  

In [11]:
params = {                                                                                                                                                                   
        "MODEL": 'non-static',                                                                                                                                                  
        "MAX_SENT_LEN": 40,                                                                           
        "BATCH_SIZE": 100,                                                                                                                                                       
        "WORD_DIM": 300,                                                                                                                                                         
        "VOCAB_SIZE": len(TEXT.vocab),                                                                                                                                        
        "HIDDEN_SIZE": 128,                                                                                                                                      
        "FILTERS": [3,4,5],                                                                                                                                                      
        "FILTER_NUM": [100,100,100],                                                                                                                                             
        "DROPOUT_PROB": 0.5,                                                                                                                                                     
        "NORM_LIMIT": 3,                                                                                                                                                         
        "GPU": -1,                                                                                                                                                      
        "model_prefix":'cnn',
        "WV_MATRIX":wv_matrix
    }                                                                                                                                          

In [12]:
def to_var(x):                                                                                                                                                                   
    if torch.cuda.is_available():                                                                                                                                                
        x = x.cuda()                                                                                                                                                             
    return Variable(x)                                                                                                                                                           


In [13]:
class FusionEncoder(nn.Module):
    def __init__(self, input_fnn_size, fnn_hidden_size, hdam_hidden_size, output_hidden_size, cnn_params):
        super(FusionEncoder, self).__init__()
        self.network_view1=CustomNet(input_fnn_size, fnn_hidden_size)
        self.network_view2=CustomHDAMVAE(n_classes=hdam_hidden_size, return_softmax=False)
        self.network_view3=CustomCNN(**cnn_params)
        self.fc1 = nn.Linear(fnn_hidden_size+hdam_hidden_size+cnn_params['HIDDEN_SIZE'], output_hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(output_hidden_size, output_hidden_size)
        self.b11=nn.BatchNorm1d(input_fnn_size)
    
    def forward(self, x):
        xb=self.b11(x[0])
        out_view1=self.network_view1(xb)
        self.network_view2.training=self.training
        out_view2, mu, log_var =self.network_view2(x[1])
        self.network_view3.training=self.training
        out_view3=self.network_view3(x[2])
        concat_views=torch.cat((out_view1, out_view2, out_view3),1)
        out = self.fc1(concat_views)
        out = self.relu(out)
        out = self.fc2(out)
        return out, mu, log_var

In [14]:
class FusionDecoder(nn.Module):
    def __init__(self, latent_hidden_size, num_classes):
        super(FusionDecoder, self).__init__()
        self.fc1 = nn.Linear(latent_hidden_size, latent_hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(latent_hidden_size, num_classes) 
        self.b11=nn.BatchNorm1d(latent_hidden_size)
    
    def forward(self, x):
        out = self.b11(x)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [15]:
class FusionAutoEncoder(nn.Module):
    def __init__(self, input_fnn_size, fnn_hidden_size, hdam_hidden_size, output_hidden_size, cnn_params, num_classes):
        super(FusionAutoEncoder, self).__init__()
        self.encoder = FusionEncoder(input_fnn_size, fnn_hidden_size, hdam_hidden_size, 2*output_hidden_size, cnn_params)
        self.decoder = FusionDecoder(output_hidden_size, num_classes)
        
    def reparameterize(self, mu, log_var):                                                                                                                                       
        """"z = mean + eps * sigma where eps is sampled from N(0, 1)."""                                                                                                         
        eps = to_var(torch.randn(mu.size(0), mu.size(1)))                                                                                                                        
        z = mu + eps * torch.exp(log_var/2)    # 2 for convert var to std                                                                                                        
        return z                                                                                                                                                                 
                                                                                                                                                                                 
    def forward(self, x):                                                                                                                                                        
        h, hdam_mu, hdam_log_var = self.encoder(x)  
        mu, log_var = torch.chunk(h, 2, dim=1)  # mean and log variance.                                                                                                         
        z = self.reparameterize(mu, log_var)                                                                                                                                     
        out = self.decoder(z)                                                                                                                                                    
        return F.log_softmax(out), mu, log_var, hdam_mu, hdam_log_var                                                                                                                                                                   

In [16]:
my_model=FusionAutoEncoder(128, 128, 128, 128, params, 3)

In [17]:
if use_cuda:
    my_model=my_model.cuda()

In [18]:
my_model

FusionAutoEncoder(
  (encoder): FusionEncoder(
    (network_view1): CustomNet(
      (fc1): Linear(in_features=128, out_features=128, bias=True)
      (relu): ReLU()
      (fc2): Linear(in_features=128, out_features=128, bias=True)
    )
    (network_view2): CustomHDAMVAE(
      (word_attn_model): AttentionWordRNN(
        (lookup): Embedding(100000, 300)
        (word_gru): GRU(300, 100, bidirectional=True)
        (softmax_word): Softmax()
      )
      (sent_attn_model): AttentionSentRNNVAE(
        (sent_gru): GRU(200, 200, bidirectional=True)
        (final_linear): Linear(in_features=200, out_features=128, bias=True)
        (softmax_sent): Softmax()
        (final_softmax): Softmax()
      )
    )
    (network_view3): CustomCNN(
      (embedding): Embedding(55072, 300, padding_idx=55071)
      (conv_0): Conv1d(1, 100, kernel_size=(900,), stride=(300,))
      (conv_1): Conv1d(1, 100, kernel_size=(1200,), stride=(300,))
      (conv_2): Conv1d(1, 100, kernel_size=(1500,), stride=(3

In [20]:
for b in train_iterator:
    break

In [27]:
my_model.encoder.network_view2(get_examples(b)[1])

custom_hdam.py:96: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  word_attn_norm = self.softmax_word(word_attn.transpose(1,0))


RuntimeError: input must have 3 dimensions, got 2

In [19]:
def get_kl(mu, log_var):
    klloss= torch.mean(torch.sum(0.5 * (mu**2 + torch.exp(log_var) - log_var -1), 1))
    return klloss

In [20]:
def train_vae(model, optimizer, train_dataset, model_dir, model_prefix, num_epochs, get_examples, get_targets, lr=0.01, max_norm=None, compute_metric=None, eval_data=None, plot_every=50, strict_batch=False):                                                                                                                                                       
    # Always feed examples batch first                                                                                                                                           
    epoch_losses=[]                                                                                                                                                              
    metrics=[]                                                                                                                                                                   
    training_metrics=[]                                                                                                                                                          
    mean_losses=[]                                                                                                                                                               
    train_dataset.iterations=0                                                                                                                                                   
    save_every=1                                                                                                                                                                 
    parameters = filter(lambda p: p.requires_grad, model.parameters())                                                                                                           
    loss_function=nn.NLLLoss()                                                                                                                                                   
    for epoch in np.arange(0,num_epochs):                                                                                                                                        
        batch_losses=[]                                                                                                                                                          
        mean_losses=[]                                                                                                                                                           
        for i, b in enumerate(train_dataset):                                                                                                                                    
            if strict_batch and (b.batch_size != train_dataset.batch_size):                                                                                                      
                continue                                                                                                                                                         
            model.train()                                                                                                                                                        
            model.zero_grad()                                                                                                                                                    
            output, mu, log_var, hdam_mu, hdam_log_var=model(get_examples(b))                                                                                                                                        
            targets=get_targets(b)                                                                                                                                               
            likelihood_loss = loss_function(output, targets)
            kl_loss=get_kl(mu, log_var)
            hdam_loss=get_kl(hdam_mu, hdam_log_var)
            total_loss=likelihood_loss + 0.01*kl_loss+0.001*hdam_loss
            print total_loss.data[0], likelihood_loss.data[0], kl_loss.data[0], hdam_loss.data[0]
            batch_losses.append(total_loss.data[0])                                                                                                                                    
            if (i%plot_every==0):                                                                                                                                                
                mean_losses.append(np.mean(batch_losses))                                                                                                                        
                yvals=np.array(mean_losses)                                                                                                                                      
                xvals=np.arange(0, len(yvals))                                                                                                                                   
                vis.line(Y=yvals, X=xvals, win='batch_loss', opts={'title':'batch_loss'})                                                                                        
            total_loss.backward()                                                                                                                                                      
            if max_norm is not None:                                                                                                                                             
                nn.utils.clip_grad_norm(parameters, max_norm=max_norm)                                                                                                           
            optimizer.step()                                                                                                                                                     
                                                                                                                                                                                 
        epoch_losses.append(np.mean(batch_losses))                                                                                                                               
        if compute_metric is not None:                                                                                                                                           
            tmetric=compute_metric(train_dataset, model, get_examples, get_targets)                                                                                              
            training_metrics.append(tmetric)                                                                                                                                     
            vis.line(np.array(training_metrics), win='training_metric', opts={'title':'training_metric'})                                                                        
            metric=compute_metric(eval_data, model, get_examples, get_targets)                                                                                                   
            metrics.append(metric)                                                                                                                                               
            vis.line(Y=np.array(metrics), X=np.arange(0, len(np.array(metrics))), win='metric', opts={'title':'metric'})                                                         
            vis.line(Y=np.array(epoch_losses), X=np.arange(0, len(np.array(metrics))), win='loss', opts={'title':'loss'})                                                        
        torch.save(model.state_dict(), "{}/{}_{}.dict".format(model_dir, model_prefix, int(epoch)))                                                                              
    return epoch_losses[-1]            

In [ ]:
optimizer = optim.Adadelta(filter(lambda p: p.requires_grad, my_model.parameters()), lr=1.0)

In [ ]:
train_vae(my_model, optimizer, train_iterator, '/mnt/sshd/vvkulkarni/fakenews_nn_models/vae_std_gauss_attn/', 'vae_std_gauss_attn', 5,  get_examples, get_targets, lr=0.1, max_norm=None, compute_metric=None, eval_data=val_iterator)

1.09893202782 1.09482586384 0.26179921627 1.48818874359
1.10495090485 1.10033726692 0.250787734985 2.10569643974
1.12733221054 1.12311983109 0.237090736628 1.84140563011
1.06069874763 1.05687820911 0.217010840774 1.6504881382
1.15713071823 1.15310835838 0.222701206803 1.79526793957
1.17215895653 1.16831088066 0.224200949073 1.60615873337
1.09699273109 1.09332716465 0.208028867841 1.58522737026
1.01284265518 1.00933766365 0.196828603745 1.53673815727
1.15013206005 1.14653003216 0.200257301331 1.59937942028
1.16921675205 1.16583991051 0.197928875685 1.39766013622
1.1489123106 1.14548969269 0.188274383545 1.53985059261
1.1373090744 1.13395631313 0.188969969749 1.46302556992
1.05769586563 1.05453372002 0.182414993644 1.33795166016
1.11177825928 1.10858845711 0.177874043584 1.41113185883
1.14496815205 1.1419481039 0.179036155343 1.22964465618
1.0587669611 1.05566740036 0.173264592886 1.36694228649
1.09003043175 1.08674228191 0.160062432289 1.68748903275
1.14517235756 1.14189481735 0.1643349

1.13900852203 1.11085140705 2.7353246212 0.803859472275
1.00607919693 0.986951649189 1.82587969303 0.86878734827
0.960458397865 0.93265080452 2.70097994804 0.797819912434
0.899978876114 0.873034834862 2.61759018898 0.768135249615
1.00107324123 0.967038750648 3.32665419579 0.767936944962
0.966722548008 0.954855501652 1.10793614388 0.787649571896
1.12634551525 1.10567140579 1.99134767056 0.760630071163
1.03930008411 1.02096045017 1.7549841404 0.789819419384
0.888615787029 0.855561435223 3.19942688942 1.06004142761
1.10254621506 1.05563151836 4.62110853195 0.703560352325
1.02597975731 0.999144017696 2.58963894844 0.939380824566
0.997835040092 0.981365382671 1.54214894772 1.04814422131
1.05204701424 1.03907847404 1.22575700283 0.711004257202
0.958790779114 0.933824360371 2.42278146744 0.738635420799
0.966952443123 0.919654905796 4.62210988998 1.0764850378
1.07735157013 1.04056739807 3.60401582718 0.743996500969
0.980076789856 0.955128490925 2.4057598114 0.890734314919
0.873185575008 0.8372

1.12119698524 1.08661806583 3.38646125793 0.714343607426
1.00154399872 0.977203428745 2.36464977264 0.694068193436
0.888871371746 0.850272774696 3.78450775146 0.753520667553
0.749295055866 0.720191001892 2.82263803482 0.877670764923
0.890851616859 0.855575084686 3.45899915695 0.68650662899
1.1506267786 1.12251913548 2.74563622475 0.651187062263
1.11715829372 1.08613908291 3.03416466713 0.677558004856
1.0032389164 0.982759416103 1.97797310352 0.699812948704
0.878911793232 0.849653482437 2.86383032799 0.620030879974
0.90973418951 0.883759140968 2.53665709496 0.608512103558
0.92393964529 0.892884671688 3.04721450806 0.582820832729
0.895126461983 0.855896949768 3.85786414146 0.650870084763
0.935267031193 0.890290975571 4.43150806427 0.660966813564
0.878454566002 0.825323879719 5.24006795883 0.730059742928
0.813331902027 0.763585686684 4.90656614304 0.680571198463
0.990056753159 0.944510996342 4.47846221924 0.761141955853
0.954639077187 0.931610763073 2.22108507156 0.817470252514
0.83924740

1.06181001663 1.00923192501 5.1836681366 0.741379737854
0.97790491581 0.931608378887 4.57318735123 0.564682126045
0.879630923271 0.829002737999 4.97987699509 0.829380452633
0.804524838924 0.760558366776 4.33773469925 0.589150846004
1.02691757679 0.987146377563 3.93031406403 0.467980563641
1.15855693817 1.13204264641 2.60379433632 0.476313471794
1.02232563496 0.98519796133 3.66192173958 0.508429884911
0.92256718874 0.892883181572 2.91688156128 0.515160918236
0.956841468811 0.926962196827 2.94139790535 0.465301781893
1.00052261353 0.970448791981 2.96122860909 0.461596220732
0.938953220844 0.911761820316 2.67157673836 0.475671589375
0.933733642101 0.893753647804 3.95272397995 0.45272949338
0.965111672878 0.916332483292 4.83253669739 0.453836679459
0.90256267786 0.86463034153 3.74836587906 0.448688387871
0.779251456261 0.739532887936 3.92449402809 0.473637402058
0.859581947327 0.820973455906 3.8167090416 0.44144192338
0.914421200752 0.863282620907 5.06978654861 0.440718740225
1.17712879181

0.913157045841 0.874133348465 3.85191202164 0.504548788071
0.853657782078 0.811231315136 4.18960189819 0.530443191528
0.864161550999 0.825863838196 3.76549768448 0.642728328705
0.973963618279 0.950643241405 2.26829648018 0.637404441833
0.900965332985 0.864628493786 3.57859039307 0.550953090191
0.864214777946 0.824463307858 3.91984391212 0.552989542484
0.898636162281 0.863831341267 3.43397641182 0.46500903368
0.878988802433 0.800138056278 7.82738208771 0.576923489571
0.791431963444 0.744664132595 4.61507987976 0.617019712925
0.986067056656 0.940088689327 4.54784965515 0.499819904566
0.877097606659 0.842975258827 3.35967540741 0.525571167469
0.922336041927 0.88876324892 3.30359792709 0.53681409359
1.09759426117 1.072083354 2.49549198151 0.555963635445
0.966974079609 0.939761281013 2.66330289841 0.579777777195
1.06295335293 1.03329098225 2.89728879929 0.689461529255
0.886915266514 0.854778409004 3.15657901764 0.571067035198
0.984757602215 0.966462850571 1.76374363899 0.657329380512
0.9592

...

0.405037969351 0.357707589865 4.30927324295 4.23766279221
0.402843594551 0.351639330387 4.69777917862 4.2264881134
0.362352013588 0.314788788557 4.33206939697 4.24252653122
0.361777186394 0.308168292046 4.92531108856 4.35579967499
0.394357770681 0.34128588438 4.92721319199 3.79978060722
0.328837305307 0.285639435053 3.92658829689 3.93200683594
0.39345857501 0.345024913549 4.41199302673 4.31372976303
0.324905455112 0.274238735437 4.69863176346 3.68041372299
0.413138329983 0.362916499376 4.56737661362 4.54808282852
0.317710042 0.258634030819 5.53179168701 3.75811314583
0.471915125847 0.414668291807 5.31234836578 4.12337303162
0.484307825565 0.429465055466 5.09921360016 3.85063505173
0.616747140884 0.564950525761 4.80619621277 3.73465871811
0.520653605461 0.470631688833 4.58174800873 4.20447015762
0.247285470366 0.196068525314 4.71749687195 4.04198026657
0.401205331087 0.3519333601 4.510014534 4.17183256149
0.299828618765 0.240746930242 5.492664814 4.1550450325
0.500456213951 0.4454123973

0.393077462912 0.34393632412 4.52261924744 3.91493868828
0.54288995266 0.493110120296 4.57592391968 4.02058124542
0.413438826799 0.365291953087 4.41092586517 4.0376162529
0.483424514532 0.439135432243 4.05085849762 3.78051900864
0.543045341969 0.497629076242 4.16166448593 3.7996404171
0.494769245386 0.45048931241 4.03176212311 3.96230173111
0.444117248058 0.400126546621 4.00208711624 3.96982264519
0.58066624403 0.540210962296 3.6744761467 3.7105345726
0.473528891802 0.426652491093 4.32079219818 3.66849851608
0.439662992954 0.394401907921 4.13265371323 3.93455886841
0.480779081583 0.435956448317 4.10059165955 3.81672763824
0.328085839748 0.279468983412 4.45703840256 4.0464720726
0.422431230545 0.374903082848 4.35070133209 4.02114677429
0.625330448151 0.581533670425 3.99962615967 3.80052661896
0.385659396648 0.336418658495 4.54012393951 3.8395011425
0.537667155266 0.494078755379 3.98627448082 3.725659132
0.548371732235 0.503989696503 4.07210969925 3.66088747978
0.352787852287 0.308226108

0.587940216064 0.540017724037 4.41451120377 3.77736592293
0.466914862394 0.411495387554 5.1283826828 4.13563632965
0.401768833399 0.351940065622 4.590883255 3.9199385643
0.285799056292 0.231536284089 4.98920965195 4.37067747116
0.441008418798 0.392057955265 4.52633142471 3.68714666367
0.437012076378 0.385798543692 4.69218111038 4.29171657562
0.317658394575 0.27027246356 4.33130121231 4.07294845581
0.315978825092 0.270380884409 4.13024759293 4.29546260834
0.58449357748 0.539684593678 4.06152868271 4.19372606277
0.346249580383 0.30103635788 4.12291288376 3.98408746719
0.43522977829 0.386402159929 4.52870321274 3.54056429863
0.472343444824 0.430487990379 3.8096575737 3.75888299942
0.503589808941 0.459293723106 4.0808134079 3.48793387413
0.478026747704 0.429293632507 4.47198152542 4.01329183578
0.552696287632 0.504417479038 4.470515728 3.57365584373
0.412604689598 0.372627586126 3.60173726082 3.95971679688
0.467469155788 0.422061920166 4.18132209778 3.59401655197
0.506361186504 0.456971734

0.615765869617 0.564486980438 4.75475072861 3.73135399818
0.36442604661 0.313162565231 4.76972532272 3.56624507904
0.487354010344 0.430127024651 5.36200332642 3.60693836212
0.575262844563 0.530929327011 4.04794692993 3.85402750969
0.586781919003 0.545179724693 3.77218985558 3.88034629822
0.478375971317 0.436295032501 3.85492062569 3.53173589706
0.461021780968 0.415804356337 4.1029253006 4.18818855286
0.427151173353 0.381451666355 4.20223665237 3.67711877823
0.487748801708 0.442446112633 4.14045000076 3.89819145203
0.425345420837 0.381800323725 3.92892241478 4.25586795807
0.383195310831 0.339668750763 3.99971914291 3.52936148643
0.50234234333 0.453623592854 4.49384498596 3.78029251099
0.425073981285 0.381049484015 4.05293798447 3.49511933327
0.429005980492 0.387148708105 3.80506181717 3.80663776398
0.392579376698 0.345452994108 4.30978345871 4.02855205536
0.317441791296 0.27234005928 4.12448740005 3.85685515404
0.35552534461 0.307590246201 4.4491648674 3.44345402718
0.396373510361 0.345

0.4533604375455199

In [32]:
def get_f1_on(dataset_iterator, model, get_examples, get_targets):                                                                                                               
    all_preds=[]                                                                                                                                                                 
    all_targets=[]                                                                                                                                                               
    model.training=False                                                                                                                                                         
    for e in dataset_iterator:                                                                                                                                                   
        model.zero_grad()                                                                                                                                                        
        output,_,_,_,_=model(get_examples(e))                                                                                                                                            
        classix=list(np.argmax(output.cpu().data.numpy(), axis=1))                                                                                                               
        targets=get_targets(e).cpu().data.numpy()                                                                                                                                
        all_preds.extend(classix)                                                                                                                                                
        all_targets.extend(targets)                                                                                                                                              
    model.training=True                                                                                                                                                          
    return f1_score(all_targets, all_preds, average='weighted')                                                                                                                  

In [33]:
get_f1_on(test_iterator, my_model, get_examples, get_targets)

0.8029463835580358

In [ ]:
get_f1_on(train_iterator, my_model, get_examples, get_targets)

In [ ]:
def get_preds_on(dataset_iterator, model, get_examples):                                                                                                                         
    results=[]                                                                                                                                                                   
    model.training=False                                                                                                                                                         
    for e in dataset_iterator:                                                                                                                                                   
        model.zero_grad()                                                                                                                                                        
        output,_,_,_,_=model(get_examples(e))                                                                                                                                            
        for pred, ix in zip(output, e.index):                                                                                                                                    
            results.append((ix.cpu().data.numpy()[0], pred.cpu().data.numpy()))                                                                                                  
                                                                                                                                                                                 
    model.training=True                                                                                                                                                          
    return results                                                                                                                                                               


In [ ]:
train_preds=get_preds_on(train_iterator, my_model, get_examples)
test_preds=get_preds_on(test_iterator, my_model, get_examples)
val_preds=get_preds_on(val_iterator, my_model, get_examples)
import pickle
pickle.dump((train_preds, val_preds, test_preds),open('/mnt/sshd/vvkulkarni/fakenews_nn_models/preds_std_vae_attn.pkl','wb'))

In [52]:
my_model.load_state_dict(torch.load('/mnt/sshd/vvkulkarni/fakenews_nn_models/vae_std_gauss_attn/vae_std_gauss_attn_4.dict'))

In [ ]:
my_model=my_model.cuda()
get_f1_on(test_iterator, my_model, get_examples, get_targets)

In [ ]:
def get_f1_on(dataset_iterator, model, get_examples, get_targets):                                                                                                               
    all_preds=[]                                                                                                                                                                 
    all_targets=[]                                                                                                                                                               
    model.training=False                                                                                                                                                         
    for e in dataset_iterator:                                                                                                                                                   
        model.zero_grad()                                                                                                                                                        
        output,_,_,_,_=model(get_examples(e)) 
        print output
        classix=list(np.argmax(output.cpu().data.numpy(), axis=1))                                                                                                               
        targets=get_targets(e).cpu().data.numpy()                                                                                                                                
        all_preds.extend(classix)                                                                                                                                                
        all_targets.extend(targets)                                                                                                                                              
    model.training=True                                                                                                                                                          
    return f1_score(all_targets, all_preds, average='weighted')                                                                                                                  

In [55]:
def get_preds_report(dataset_iterator, model, get_examples, get_targets):                                                                                                               
    all_preds=[]                                                                                                                                                                 
    all_targets=[] 
    all_indices=[]
    model.training=False                                                                                                                                                         
    for e in dataset_iterator:                                                                                                                                                   
        model.zero_grad()                                                                                                                                                        
        output,_,_,_,_=model(get_examples(e)) 
        classix=list(np.argmax(output.cpu().data.numpy(), axis=1))                                                                                                               
        targets=get_targets(e).cpu().data.numpy()
        indices=e.index.cpu().data.numpy()
        all_preds.extend(classix)                                                                                                                                                
        all_targets.extend(targets)
        all_indices.extend(indices)
    model.training=True                                                                                                                                                          
    return all_targets, all_preds, all_indices                                                                                                                

In [56]:
train_preds=get_preds_report(train_iterator, my_model, get_examples, get_targets)
test_preds=get_preds_report(test_iterator, my_model, get_examples, get_targets)
val_preds=get_preds_report(val_iterator, my_model, get_examples, get_targets)
import pickle
pickle.dump((train_preds, val_preds, test_preds),open('/mnt/sshd/vvkulkarni/fakenews_nn_models/preds_our_model_attn_vae_clf_report.pkl','wb'))
from sklearn.metrics import classification_report
print classification_report(test_preds[0], test_preds[1])

             precision    recall  f1-score   support

          0       0.79      0.80      0.80      3362
          1       0.87      0.78      0.82      2855
          2       0.72      0.80      0.76      2110

avg / total       0.80      0.80      0.80      8327



In [57]:
print classification_report(test_preds[0], test_preds[1], digits=4)

             precision    recall  f1-score   support

          0     0.7904    0.8040    0.7971      3362
          1     0.8750    0.7793    0.8244      2855
          2     0.7179    0.8043    0.7586      2110

avg / total     0.8010    0.7956    0.7967      8327

